In [11]:
import sys
sys.path = ['', '/Applications/Spyder-Py2.app/Contents/Resources', '/Applications/Spyder-Py2.app/Contents/Resources/lib/python27.zip', '/Applications/Spyder-Py2.app/Contents/Resources/lib/python2.7', '/Applications/Spyder-Py2.app/Contents/Resources/lib/python2.7/plat-darwin', '/Applications/Spyder-Py2.app/Contents/Resources/lib/python2.7/plat-mac', '/Applications/Spyder-Py2.app/Contents/Resources/lib/python2.7/plat-mac/lib-scriptpackages', '/Applications/Spyder-Py2.app/Contents/Resources/lib/python2.7/lib-tk', '/Applications/Spyder-Py2.app/Contents/Resources/lib/python2.7/lib-old', '/Applications/Spyder-Py2.app/Contents/Resources/lib/python2.7/lib-dynload', '/Applications/Spyder-Py2.app/Contents/Resources/lib/python2.7/site-packages.zip', '/Applications/Spyder-Py2.app/Contents/Resources/lib/python2.7/site-packages', '/Library/Python/2.7/site-packages/']

## E.G.1 Gaussian HMM of stock data

In [21]:
from __future__ import print_function

In [44]:
import datetime

import numpy as np
from matplotlib import cm, pyplot as plt
from matplotlib.dates import YearLocator, MonthLocator
try:
    from matplotlib.finance import quotes_historical_yahoo_ochl
except ImportError:
    # For Matplotlib prior to 1.5.
    from matplotlib.finance import (
        quotes_historical_yahoo as quotes_historical_yahoo_ochl
    )

from hmmlearn.hmm import GaussianHMM

### Get quotes from Yahoo! finance

In [58]:
quotes = quotes_historical_yahoo_ochl(
    "INTC", datetime.date(1995, 1, 1), datetime.date(2012, 1, 6))

# Unpack quotes
dates = np.array([q[0] for q in quotes], dtype=int)
close_v = np.array([q[2] for q in quotes])
volume = np.array([q[5] for q in quotes])[1:]

# Take diff of close value. Note that this makes
# ``len(diff) = len(close_t) - 1``, therefore, other quantities also
# need to be shifted by 1.
diff = np.diff(close_v)
dates = dates[1:]
close_v = close_v[1:]

# Pack diff and volume for training.
X = np.column_stack([diff, volume])

In [59]:
print (diff.shape)
print (dates.shape)
print (close_v.shape)
print (X.shape)
print ('X is the observation sequence that is made of two variables, diff and volume')

(4285,)
(4285,)
(4285,)
(4285, 2)
X is the observation sequence that is made of two variables, diff and volume


In [60]:
X

array([[ -5.34500000e-03,   4.18656000e+07],
       [  2.13800000e-02,   6.04800000e+07],
       [  3.74160000e-02,   5.63168000e+07],
       ..., 
       [  4.79110000e-01,   4.75040000e+07],
       [  2.43757000e-01,   4.94906000e+07],
       [ -1.26081000e-01,   3.63435000e+07]])

### Fit an Gaussian HMM given the observed sequence of two variables

In [61]:
print("fitting to HMM and decoding ...", end="")

# Make an HMM instance and execute fit
# n_components = Number of states
# n_iter = Maximum number of iterations to perform when fitting using Expectation Maximization
model = GaussianHMM(n_components=4, covariance_type="diag", n_iter=1000).fit(X)

# Predict the optimal sequence of internal hidden state
hidden_states = model.predict(X)

print("done")

fitting to HMM and decoding ...done


In [53]:
hidden_states.shape

(4285,)

In [73]:
hidden_states

array([2, 2, 2, ..., 1, 1, 1])

### Print trained parameters and plot

In [23]:
print("Transition matrix")
print(model.transmat_)
print()

print("Means and vars of each hidden state")
for i in range(model.n_components):
    print("{0}th hidden state".format(i))
    print("mean = ", model.means_[i])
    print("var = ", np.diag(model.covars_[i]))
    print()

fig, axs = plt.subplots(model.n_components, sharex=True, sharey=True)
colours = cm.rainbow(np.linspace(0, 1, model.n_components))
for i, (ax, colour) in enumerate(zip(axs, colours)):
    # Use fancy indexing to plot data in each state.
    mask = hidden_states == i
    ax.plot_date(dates[mask], close_v[mask], ".-", c=colour)
    ax.set_title("{0}th hidden state".format(i))

    # Format the ticks.
    ax.xaxis.set_major_locator(YearLocator())
    ax.xaxis.set_minor_locator(MonthLocator())

    ax.grid(True)

plt.show()

Transition matrix
[[  9.79217668e-01   3.55711000e-15   2.72107464e-03   1.80612577e-02]
 [  1.21578610e-12   7.73505011e-01   1.85143008e-01   4.13519812e-02]
 [  3.25251106e-03   1.12651307e-01   8.83405352e-01   6.90829241e-04]
 [  1.18930169e-01   4.20115152e-01   1.91501413e-18   4.60954680e-01]]

Means and vars of each hidden state
0th hidden state
mean =  [  2.33386318e-02   4.97390962e+07]
var =  [  6.97686060e-01   2.49469170e+14]

1th hidden state
mean =  [  2.12632772e-02   8.82101387e+07]
var =  [  1.18722534e-01   5.64904675e+14]

2th hidden state
mean =  [  7.69252417e-03   5.43200602e+07]
var =  [  5.02390623e-02   1.54646248e+14]

3th hidden state
mean =  [ -3.53843368e-01   1.53097547e+08]
var =  [  2.55860374e+00   5.88900261e+15]



## E.G.2 Build HMM model with given parameters. 
### Infer hidden state sequence given observations X. 

In [25]:
from hmmlearn import hmm

In [55]:
# n_components=3 means three hidden states [0, 1, 2]
model = hmm.MultinomialHMM(n_components=3)

# start probabilities for each hidden state
model.startprob_ = np.array([0.3, 0.4, 0.3])

# transitional matrix for pair-wise hidden state
model.transmat_ = np.array([[0.2, 0.6, 0.2],
                            [0.4, 0.0, 0.6],
                            [0.1, 0.2, 0.7]])

# emission probability matrix for [hidden state,observation]
model.emissionprob_ = np.array([[0.2, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
                                [0.1, 0.1, 0.1, 0.1, 0.2, 0.1, 0.1, 0.1, 0.1],
                                [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2]])

# Predict the optimal sequence of internal hidden state
X = np.atleast_2d([0,3,4,5,6,7,1,2,3,8]).T
print ('log prob of the produced sequence:',model.decode(X)[0])
print ('hidden state sequence for each observation in sample X:',model.decode(X)[1])

log prob of the produced sequence: -26.3619052282
hidden state sequence for each observation in sample X: [0 1 2 2 2 2 2 2 2 2]


### Generate sample sequence 

In [63]:
# generate sample sequence for the HMM model
x, z = model.sample(100)
print (x.shape)
print (z.shape)

(100, 2)
(100,)
